# Quart + Jinja2 + Three.js Chart Demo

This notebook demonstrates how to:
- Use **Pandas** to create sample data
- Build an async web server with **Quart**
- Render HTML templates with **Jinja2**
- Visualize data with **Three.js** 3D charts
- Expose the server via **Colab's built-in port serving**

**Goal**: Show a simple, clean pattern for building interactive 3D visualizations.

## 1. Install Dependencies

Install Quart (async web framework), Pandas, and NumPy.

In [ ]:
!pip install -q quart pandas numpy

## 2. Create Sample Data

Generate simple sales data using Pandas.

In [ ]:
import pandas as pd
import numpy as np

# Create sample quarterly sales data for different products
data = {
    'Product': ['Product A', 'Product B', 'Product C', 'Product D'],
    'Q1': [45, 60, 38, 52],
    'Q2': [58, 72, 45, 61],
    'Q3': [62, 68, 51, 58],
    'Q4': [71, 85, 59, 67]
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df)

# Convert to format suitable for JavaScript
chart_data = df.to_dict('records')
print("\nData for chart:", chart_data)

## 3. Set Up Quart Application

Create a Quart app with a single route that renders our chart.

In [ ]:
from quart import Quart, render_template_string
import json

app = Quart(__name__)

# Jinja2 template with Three.js chart
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>3D Bar Chart - Quart + Three.js</title>
    <style>
        body {
            margin: 0;
            padding: 20px;
            font-family: Arial, sans-serif;
            background: #f0f0f0;
        }
        h1 {
            text-align: center;
            color: #333;
        }
        #chart-container {
            width: 100%;
            height: 600px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .info {
            text-align: center;
            margin-top: 10px;
            color: #666;
        }
    </style>
</head>
<body>
    <h1>Quarterly Sales - 3D Bar Chart</h1>
    <div id="chart-container"></div>
    <div class="info">Drag to rotate • Scroll to zoom • Right-click to pan</div>

    <!-- Include Three.js from CDN -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>
        // Data passed from Python via Jinja2
        const chartData = {{ data | tojson }};
        
        // Set up the scene
        const container = document.getElementById('chart-container');
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0xffffff);
        
        // Set up camera
        const camera = new THREE.PerspectiveCamera(
            75,
            container.clientWidth / container.clientHeight,
            0.1,
            1000
        );
        camera.position.set(15, 15, 15);
        camera.lookAt(0, 0, 0);
        
        // Set up renderer
        const renderer = new THREE.WebGLRenderer({ antialias: true });
        renderer.setSize(container.clientWidth, container.clientHeight);
        container.appendChild(renderer.domElement);
        
        // Add lights
        const ambientLight = new THREE.AmbientLight(0x404040, 1.5);
        scene.add(ambientLight);
        
        const directionalLight = new THREE.DirectionalLight(0xffffff, 0.8);
        directionalLight.position.set(10, 10, 10);
        scene.add(directionalLight);
        
        // Create 3D bars from data
        const quarters = ['Q1', 'Q2', 'Q3', 'Q4'];
        const colors = [0x3498db, 0xe74c3c, 0x2ecc71, 0xf39c12]; // Blue, Red, Green, Orange
        
        chartData.forEach((product, productIndex) => {
            quarters.forEach((quarter, quarterIndex) => {
                const value = product[quarter];
                const height = value / 10; // Scale height
                
                // Create bar geometry
                const geometry = new THREE.BoxGeometry(1.5, height, 1.5);
                const material = new THREE.MeshPhongMaterial({ 
                    color: colors[productIndex],
                    shininess: 100
                });
                const bar = new THREE.Mesh(geometry, material);
                
                // Position the bar
                const x = productIndex * 4 - 6;
                const z = quarterIndex * 4 - 6;
                bar.position.set(x, height / 2, z);
                
                scene.add(bar);
                
                // Add edge lines for better visibility
                const edges = new THREE.EdgesGeometry(geometry);
                const line = new THREE.LineSegments(
                    edges,
                    new THREE.LineBasicMaterial({ color: 0x000000, linewidth: 1 })
                );
                line.position.copy(bar.position);
                scene.add(line);
            });
        });
        
        // Add grid helper for reference
        const gridHelper = new THREE.GridHelper(20, 20, 0xcccccc, 0xeeeeee);
        scene.add(gridHelper);
        
        // Add axes helper
        const axesHelper = new THREE.AxesHelper(10);
        scene.add(axesHelper);
        
        // Mouse controls for rotation
        let isDragging = false;
        let previousMousePosition = { x: 0, y: 0 };
        let rotation = { x: 0, y: 0 };
        
        container.addEventListener('mousedown', (e) => {
            isDragging = true;
            previousMousePosition = { x: e.clientX, y: e.clientY };
        });
        
        container.addEventListener('mousemove', (e) => {
            if (isDragging) {
                const deltaX = e.clientX - previousMousePosition.x;
                const deltaY = e.clientY - previousMousePosition.y;
                
                rotation.y += deltaX * 0.01;
                rotation.x += deltaY * 0.01;
                
                previousMousePosition = { x: e.clientX, y: e.clientY };
            }
        });
        
        container.addEventListener('mouseup', () => {
            isDragging = false;
        });
        
        // Zoom with mouse wheel
        container.addEventListener('wheel', (e) => {
            e.preventDefault();
            camera.position.multiplyScalar(1 + e.deltaY * 0.001);
        });
        
        // Animation loop
        function animate() {
            requestAnimationFrame(animate);
            
            // Apply rotation
            camera.position.x = 15 * Math.cos(rotation.y) * Math.cos(rotation.x);
            camera.position.y = 15 * Math.sin(rotation.x) + 10;
            camera.position.z = 15 * Math.sin(rotation.y) * Math.cos(rotation.x);
            camera.lookAt(0, 3, 0);
            
            renderer.render(scene, camera);
        }
        
        // Handle window resize
        window.addEventListener('resize', () => {
            camera.aspect = container.clientWidth / container.clientHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(container.clientWidth, container.clientHeight);
        });
        
        // Start animation
        animate();
    </script>
</body>
</html>
'''

@app.route('/')
async def index():
    """Main route that renders the 3D chart with data"""
    return await render_template_string(HTML_TEMPLATE, data=chart_data)

print("✓ Quart app configured")

## 4. Set Up Port Serving

Use Colab's built-in port serving to access the Quart server.

In [ ]:
from google.colab import output
import nest_asyncio

# Allow nested async loops (required for Colab)
nest_asyncio.apply()

# Serve the app on port 5000 using Colab's built-in port serving
port = 5000
output.serve_kernel_port_as_window(port)
print(f"🌐 Server will be accessible via Colab's port serving on port {port}")
print(f"\nThe visualization will open in a new window once the server starts.")

## 5. Run the Server

Start the Quart server and display the visualization.

**Note**: This cell will run continuously. Click the link above to view the chart.

In [ ]:
import asyncio

# Run the Quart app
print("Server starting... The visualization should open in a new window.")
await app.run_task(host='0.0.0.0', port=port, debug=False)

## 6. Cleanup (Optional)

No cleanup needed - Colab's port serving will stop when the server stops.

In [ ]:
# No cleanup needed for Colab's built-in port serving
print("Server stopped. Port serving cleaned up automatically.")

---

## Summary

This notebook demonstrates:

1. **Data Pipeline**: Pandas → Python dict → JSON → JavaScript
2. **Web Framework**: Quart (async) with Jinja2 templating
3. **3D Visualization**: Three.js for interactive charts
4. **Deployment**: Colab's built-in port serving

### Key Features:
- ✅ Simple, minimal code
- ✅ Interactive 3D controls (drag to rotate, scroll to zoom)
- ✅ Clean data flow from Python to JavaScript
- ✅ Professional presentation-ready output

### Customization Ideas:
- Change the data in Step 2
- Modify colors in the Three.js code
- Add more chart types (scatter, line, etc.)
- Enhance with additional Three.js features (labels, animations, etc.)
